In [2]:
import ee
import geemap
import geopandas 
from ipyleaflet import GeoJSON
from zipfile import ZipFile
ee.Initialize()

Create a image collection from satellite sensor:

In [3]:
image = (ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4").
         select("CH4_column_volume_mixing_ratio_dry_air_bias_corrected").
         filterDate('2022-01-01', '2023-01-01'))

Map = geemap.Map()

import a shapefile and create his layer:

In [10]:
departamentos = ee.FeatureCollection('users/hmojica/data/departamentos')

Map.addLayer(departamentos, {}, 'dep')
Map

Map(bottom=4301.0, center=[4.171115454867424, -74.53091701696118], controls=(WidgetControl(options=['position'…

#### ee.ImageCollection.mean
Reduces an image collection by calculating the mean of all values at each pixel across the stack of all matching bands. Bands are matched by name.

In [25]:
band_viz = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map.addLayer(image.mean(), band_viz, 'methane')
Map.setCenter(-74.297333, 4.570868, 5);

# Se define la region de interes
xMin = -65.654297
yMin = -4.954507
xMax = -82.573242
yMax = 13.330474
region = ee.Geometry.Rectangle([
    [xMin, yMin],
    [xMax, yMax]
  ])

In [13]:
# se obtiene las coordenadas de la región dibujndo la region con un poligono
#region = Map.user_roi #.getInfo()
#print(region)


In [14]:
#Map.addLayer(image.mean(), band_viz, 'S5P Cloud')
#geemap.clip_image(Map, region, Map)


El resultado de dibujar el poligono y obtener las coordenadas es un json:

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -82.573242,
              -4.954507
            ],
            [
              -82.573242,
              13.330474
            ],
            [
              -65.654297,
              13.330474
            ],
            [
              -65.654297,
              -4.954507
            ],
            [
              -82.573242,
              -4.954507
            ]
          ]
        ]
      }
    }
  }
})


In [15]:
# Mosaic the visualization layers and display (or export).
mosaic = image.mosaic()
# Map.addLayer(mosaic, {}, 'mosaic', False)

# Create a circle by drawing a 20000 meter buffer around a point.
clipped = mosaic.clip(region)

# Display a clipped version of the mosaic.
band_viz = {
    'min': 0,
    'max': 0.95,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map.addLayer(clipped, band_viz, 'Clipped image')

# Display the map
Map

Map(center=[4.570868, -74.297333], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…